# The Epitome of Salt's Sunrise Show: Avatar: The Last Airbender (TLA)
## Author: Kevin Brar
## Guests: Nate, Josh, Jon, Michael


In [8]:
# raters
raters = ['Kevin', 'Nate', "Jon"]

In [18]:
import pandas as pd
from itertools import combinations

def read_ratings(rater):
    try:
        df = pd.read_csv(f'data/{rater}.csv')
    except FileNotFoundError:
        print(f"Error: Could not find file data/{rater}.csv. Please make sure it's in a 'data' subfolder.")
        return pd.DataFrame(columns=['Name', 'Rarity', 'Color', 'Rating'])
        
    if 'Name' in df.columns:
        df['Name'] = df['Name'].str.strip()
    else:
        print(f"Warning: 'Name' column not found in {rater}.csv")
        df['Name'] = pd.NA 
        
    rarity_map = {
        'common': 'C',
        'uncommon': 'U',
        'rare': 'R',
        'mythic': 'M'
    }
    if 'Rarity' in df.columns:
        df['Rarity'] = df['Rarity'].map(rarity_map)
    else:
        print(f"Warning: 'Rarity' column not found in {rater}.csv")
        df['Rarity'] = pd.NA 

    if 'Color' in df.columns:
        df['Color'] = df['Color'].fillna('C')
        df['Color'] = df['Color'].apply(lambda x: 'M' if isinstance(x, str) and len(x) > 1 else x)
    else:
        print(f"Warning: 'Color' column not found in {rater}.csv")
        df['Color'] = pd.NA 
        
    if 'Rating' not in df.columns:
        print(f"Warning: 'Rating' column not found in {rater}.csv")
        df['Rating'] = pd.NA 

    return df[['Name', 'Rarity', 'Color', 'Rating']]

tier_num = {'G':0., 'D-': 1., 'D': 2., 'D+': 3., 
            'C-': 4., 'C': 5., 'C+': 6., 'B-': 7., 
            'B': 8., 'B+': 9., 'A-': 10., 'A': 11., 
            'A+': 12.}

if not raters:
    print("Error: 'raters' list is empty. Please define it.")
else:
    eos = read_ratings(raters[0])
    eos.rename(columns={'Rating': raters[0]}, inplace=True)

    for rater in raters[1:]:
        rater_data = read_ratings(rater)
        
        rater_ratings = rater_data[['Name', 'Rating']].rename(columns={'Rating': rater})
        
        eos = pd.merge(eos, rater_ratings, on='Name', how='outer')

    print("--- Unrated Card Analysis ---")
    all_cards_rated = True
    for rater in raters:
        unrated_cards = eos[eos[rater].isnull()]['Name']
        if unrated_cards.empty:
            print(f"All cards were rated by {rater}.")
        else:
            all_cards_rated = False
            print(f"\nCards NOT rated by {rater}:")
            print(unrated_cards.to_string(index=False))
            
    if all_cards_rated and not eos[raters[0]].isnull().any():
        print("All raters rated all cards. Perfect match.")
        
    print("\n" + "="*30 + "\n")

    for rater in raters:
        eos[f'{rater}_num'] = eos[rater].map(tier_num)

    rater_num_cols = [f'{rater}_num' for rater in raters]
    eos_clean = eos.dropna(subset=rater_num_cols).copy()

eos = eos_clean
eos

--- Unrated Card Analysis ---
All cards were rated by Kevin.
All cards were rated by Nate.
All cards were rated by Jon.
All raters rated all cards. Perfect match.




,Name,Rarity,Color,Kevin,Nate,Jon,Kevin_num,Nate_num,Jon_num
0,Aang's Iceberg,R,W,B,B-,B-,8.0,7.0,7.0
1,Aang's Journey,C,C,C,C,D,5.0,5.0,2.0
2,"Aang, Swift Savior // Aang and La, Ocean's Fury",R,M,A,A,B,11.0,11.0,8.0
3,"Aang, at the Crossroads // Aang, Destined Savior",R,M,A-,C,B+,10.0,5.0,9.0
4,"Aang, the Last Airbender",U,W,B,C+,B,8.0,6.0,8.0
...,...,...,...,...,...,...,...,...,...
281,"Zhao, the Moon Slayer",R,R,C,C,C,5.0,5.0,5.0
282,Zuko's Conviction,U,B,C,C+,C+,5.0,6.0,6.0
283,Zuko's Exile,C,C,D,D,C+,2.0,2.0,6.0
284,"Zuko, Conflicted",R,M,B,D,B-,8.0,2.0,7.0


In [10]:
colors = ['W', 'R', 'B', 'U', 'G']

# Calculate the 'Group' average (mean rating) for each card
# This line is corrected from your screenshot
eos['Group'] = eos[[rater + '_num' for rater in raters]].sum(axis=1) / len(raters)

color_pairs = list(combinations(colors, 2))
color_pair_scores = {}

for pair in color_pairs:
    color1, color2 = pair
    
    # Find all rows that match *either* color1 or color2
    pair_rows = eos[(eos['Color'] == color1) | (eos['Color'] == color2)] 
    
    total_score = 0.0 # Default score if no cards are found
    if len(pair_rows) > 0:
        # Calculate the average AND round it to 2 decimal places
        total_score = round(pair_rows['Group'].sum() / len(pair_rows), 2)
    
    color_pair_scores[pair] = total_score

# --- NEW: Sort into a LIST of tuples, not a dict ---
sorted_color_pair_list = sorted(color_pair_scores.items(), key=lambda item: item[1], reverse=True)

# This will print the clean, sorted list you want
print("--- Average Rating by Color Pair (Sorted) ---")
for x in sorted_color_pair_list:
    print(f"Colors: {x[0][0]} & {x[0][1]} - Average Rating: {x[1]}")

--- Average Rating by Color Pair (Sorted) ---
Colors: W & G - Average Rating: 5.57
Colors: W & U - Average Rating: 5.5
Colors: W & R - Average Rating: 5.46
Colors: W & B - Average Rating: 5.4
Colors: U & G - Average Rating: 5.26
Colors: R & G - Average Rating: 5.23
Colors: B & G - Average Rating: 5.17
Colors: R & U - Average Rating: 5.16
Colors: B & U - Average Rating: 5.1
Colors: R & B - Average Rating: 5.07


## Top and Bottom Commons and Uncommons

In [11]:
eos_commons = eos[eos['Rarity'] == 'C']
top_cards_by_color = eos_commons.groupby('Color').apply(
    lambda x: x.nlargest(3, 'Group'), 
    include_groups=False
).reset_index()

print("--- Top 3 Commons by Color ---")
print(top_cards_by_color[['Name', 'Color', 'Group']].to_string(index=False))

--- Top 3 Commons by Color ---
                      Name Color    Group
                  Sold Out     B 6.666667
           Swampsnare Trap     B 5.000000
         Callous Inspector     B 4.666667
            Aang's Journey     C 4.000000
              Rumble Arena     C 4.000000
 Barrels of Blasting Jelly     C 3.333333
              Rocky Rebuke     G 6.666667
                Badgermole     G 6.000000
             Ostrich-Horse     G 6.000000
            Serpent's Pass     M 5.666667
          Sun-Blessed Peak     M 5.666667
       Airship Engine Room     M 5.333333
          Lightning Strike     R 8.000000
        Firebending Lesson     R 6.333333
  Treetop Freedom Fighters     R 6.333333
Forecasting Fortune Teller     U 6.666667
         Rowdy Snowballers     U 5.333333
             Iguana Parrot     U 5.000000
           Kyoshi Warriors     W 6.333333
        Path to Redemption     W 5.666667
         Airbending Lesson     W 5.333333


In [12]:
eos_uncommons = eos[eos['Rarity'] == 'U']
top_cards_by_color = eos_uncommons.groupby('Color').apply(lambda x: x.nlargest(3, 'Group')).reset_index(drop=True)
print(top_cards_by_color[['Name', 'Color', 'Group']].to_string(index=False))
print("--- Top 3 Uncommons by Color ---")

                    Name Color    Group
      Azula, On the Hunt     B 7.000000
           Heartless Act     B 7.000000
           Epic Downfall     B 6.333333
            Meteor Sword     C 5.333333
     Fire Nation Warship     C 4.333333
     White Lotus Hideout     C 4.333333
  Toph, the Blind Bandit     G 8.000000
          Allies at Last     G 7.666667
   Earth Kingdom General     G 6.666667
    Suki, Kyoshi Warrior     M 7.333333
           Dai Li Agents     M 7.000000
            Sun Warriors     M 7.000000
  The Cave of Two Lovers     R 7.333333
    Combustion Technique     R 7.000000
          Combustion Man     R 6.666667
 Katara, Bending Prodigy     U 7.666667
        Knowledge Seeker     U 7.333333
 Benevolent River Spirit     U 7.000000
    Earth Kingdom Jailer     W 7.666667
Aang, the Last Airbender     W 7.333333
          Master Piandao     W 7.000000
--- Top 3 Uncommons by Color ---


C:\Users\Kevbot\AppData\Local\Temp\ipykernel_18328\1248522804.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_cards_by_color = eos_uncommons.groupby('Color').apply(lambda x: x.nlargest(3, 'Group')).reset_index(drop=True)


In [13]:
eos_commons = eos[eos['Rarity'] == 'C']
top_cards_by_color = eos_commons.groupby('Color').apply(
    lambda x: x.nsmallest(3, 'Group'), 
    include_groups=False
).reset_index()

print("--- Bottom 3 Commons by Color ---")
print(top_cards_by_color[['Name', 'Color', 'Group']].to_string(index=False))

--- Bottom 3 Commons by Color ---
                     Name Color    Group
               Hog-Monkey     B 2.666667
        Azula Always Lies     B 3.000000
      Foggy Swamp Hunters     B 3.000000
        Kyoshi Battle Fan     C 3.000000
Barrels of Blasting Jelly     C 3.333333
       Bender's Waterskin     C 3.333333
              Turtle-Duck     G 1.333333
         Cycle of Renewal     G 2.666667
      Earthbending Lesson     G 2.666667
            Platypus-Bear     M 2.000000
                  Cat-Owl     M 3.666667
        Vindictive Warden     M 4.000000
       Fire Nation Cadets     R 3.000000
       Fire Nation Raider     R 3.333333
                 Mountain     R 3.333333
     Flexible Waterbender     U 3.000000
            Geyser Leaper     U 3.000000
            Otter-Penguin     U 3.000000
     Curious Farm Animals     W 3.000000
                   Plains     W 3.333333
                 Yip Yip!     W 3.333333


In [14]:
eos_uncommons = eos[eos['Rarity'] == 'U']
top_cards_by_color = eos_uncommons.groupby('Color').apply(lambda x: x.nsmallest(3, 'Group')).reset_index(drop=True)
print("--- Bottom 3 Uncommons by Color ---")
print(top_cards_by_color[['Name', 'Color', 'Group']].to_string(index=False))

--- Bottom 3 Uncommons by Color ---
                         Name Color    Group
          Northern Air Temple     B 1.333333
          Buzzard-Wasp Colony     B 3.000000
       Lo and Li, Twin Tutors     B 3.000000
             Trusty Boomerang     C 2.000000
                Energybending     C 4.000000
          Fire Nation Warship     C 4.333333
          Kyoshi Island Plaza     G 1.333333
               Sparring Dummy     G 3.000000
             Invasion Tactics     G 4.000000
Professor Zei, Anthropologist     M 3.666667
             Air Nomad Legacy     M 4.000000
                 Tolls of War     M 5.000000
       Crescent Island Temple     R 2.333333
             Price of Freedom     R 2.666667
           Jet's Brainwashing     R 3.333333
                 Master Pakku     U 2.333333
         Teo, Spirited Glider     U 3.333333
                Sokka's Haiku     U 4.000000
           Vengeful Villagers     W 3.333333
               Fancy Footwork     W 4.000000
          Southern 

C:\Users\Kevbot\AppData\Local\Temp\ipykernel_18328\3723759558.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_cards_by_color = eos_uncommons.groupby('Color').apply(lambda x: x.nsmallest(3, 'Group')).reset_index(drop=True)


## Goodest Art

In [15]:
sorted_eos = eos.sort_values(by='Group', ascending=True).copy()
print(sorted_eos[['Name', 'Group']][:3].to_string(index=False))

                   Name    Group
The Walls of Ba Sing Se 0.000000
    Northern Air Temple 1.333333
    Kyoshi Island Plaza 1.333333


## Highest Variance

In [16]:
eos['Variance'] = eos[[rater + '_num' for rater in raters]].var(axis=1)
sorted_eos = eos.sort_values(by='Variance', ascending=False).copy()
print(sorted_eos[['Name', 'Variance']][:15].to_string(index=False))

                                     Name  Variance
                           Avatar Destiny 13.000000
                    Phoenix Fleet Airship 12.333333
                         Invasion Tactics 12.000000
                        Iroh, Grand Lotus 12.000000
                                    Swamp 10.333333
                         Zuko, Conflicted 10.333333
    The Legend of Kyoshi // Avatar Kyoshi  9.333333
                          Fated Firepower  9.333333
The Legend of Yangchen // Avatar Yangchen  9.000000
                                   Island  8.333333
                                   Plains  8.333333
                                 Mountain  8.333333
                                   Forest  8.333333
                               Tiger-Seal  8.333333
                             Realm of Koh  7.000000


## Hottest takes 

In [17]:
for rater in raters:
    other_raters = [col for col in eos.columns if col.endswith('_num') and col != f'{rater}_num']
    eos[f'{rater}_mean'] = eos[other_raters].mean(axis=1)

for rater in raters:
    eos[f'{rater}_Hot_Take'] = abs(eos[f'{rater}_num'] - eos[f'{rater}_mean'])

for rater in raters:
    sorted_eos = eos.sort_values(by= rater + '_Hot_Take', ascending=False).copy()
    print(sorted_eos[['Name', rater + '_Hot_Take']][:3].to_string(index=False))
    print()

                 Name  Kevin_Hot_Take
    Iroh, Grand Lotus             6.0
       Avatar Destiny             6.0
Phoenix Fleet Airship             5.5

                                 Name  Nate_Hot_Take
                     Invasion Tactics            6.0
                     Zuko, Conflicted            5.5
The Legend of Kyoshi // Avatar Kyoshi            5.0

               Name  Jon_Hot_Take
    Fated Firepower           5.0
         Tiger-Seal           5.0
Wandering Musicians           4.5

